<a href="https://colab.research.google.com/github/JSJeong-me/GPT_Ochestration/blob/main/CSV-LangChain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain
!pip install chromadb

In [ ]:
!pip install openai
!pip install python-dotenv

In [3]:
!echo "OPENAI_API_KEY=sk-" >> .env
!source /content/.env

In [4]:
import csv
from langchain.docstore.document import Document
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

In [5]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()
# Access the API key using the variable name defined in the .env file
api_key = os.getenv("OPENAI_API_KEY")

In [6]:
# Define the columns we want to embed vs which ones we want in metadata
columns_to_embed = ["Description","Features"]
columns_to_metadata = ["Product Name","Price", "Rating","Description", "Features"]

In [7]:
# Process the CSV into the embedable content vs the metadata and put it into Document format so that we can chunk it into pieces.
docs = []
with open('TestListings.csv', newline="", encoding='utf-8-sig') as csvfile:
    csv_reader = csv.DictReader(csvfile)
    for i, row in enumerate(csv_reader):
        to_metadata = {col: row[col] for col in columns_to_metadata if col in row}
        values_to_embed = {k: row[k] for k in columns_to_embed if k in row}
        to_embed = "\n".join(f"{k.strip()}: {v.strip()}" for k, v in values_to_embed.items())
        newDoc = Document(page_content=to_embed, metadata=to_metadata)
        docs.append(newDoc)

In [8]:
# Lets split the document using Chracter splitting.
splitter = CharacterTextSplitter(separator = "\n",
                                chunk_size=500,
                                chunk_overlap=0,
                                length_function=len)
documents = splitter.split_documents(docs)

In [9]:
# import
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
from langchain_community.vectorstores import Chroma

In [10]:
# # create the open-source embedding function
# embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

# # load it into Chroma
# db = Chroma.from_documents(docs, embedding_function)

In [11]:
# import OpenAIEmbeddings
# # Generate embeddings from documents and store in a vector database
# embeddings_model = OpenAIEmbeddings()
# db = Chroma.from_documents(documents, OpenAIEmbeddings())

In [12]:
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 20.7 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=750c98a3230b28a108da2b585f8ae3ddf19bb7abe81aa50241c9d802825131f2
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [ ]:
# import
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders.csv_loader import CSVLoader


loader = CSVLoader(file_path='TestListings.csv')
documents = loader.load()



# split it into chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

# create the open-source embedding function
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

# load it into Chroma
db = Chroma.from_documents(docs, embedding_function)

# # query it
# query = "What did the president say about Ketanji Brown Jackson"
# docs = db.similarity_search(query)

# # print results
# print(docs[0].page_content)

In [15]:
# Query the vector database for information.
query = "Heart rate monitor"
docs = db.similarity_search(query)
print(docs[0].page_content)
print(docs[0].metadata)

﻿Product Name: Fitness Tracker Smartwatch
Price: 149.99
Rating: 4.5
Description: Achieve your fitness goals with our Fitness Tracker Smartwatch. Monitor your activity, heart rate, and receive notifications on your wrist.
Features: - Heart rate monitor and activity tracking.<br> - Built-in GPS for accurate workout tracking.<br> - Sleep analysis and guided breathing exercises.<br> - Receive notifications from your smartphone.<br> - Water-resistant for workouts and everyday use.<br> - Long battery life.
{'row': 5, 'source': 'TestListings.csv'}


Advanced Querying

In [16]:
from langchain.llms import OpenAI
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.base import AttributeInfo

# Metadata schema based on the values on the CSV
metadata_field_info = [
    AttributeInfo(
        name="Product Name",
        description="Name of the product",
        type="string",
    ),
    AttributeInfo(
        name="Price",
        description="The price of the product as a number. Ex. 149.99",
        type="string",
    ),
    AttributeInfo(
        name="Rating",
        description="The rating of the product as a number from 0 to 5. Ex. 4.5",
        type="string",
    ),
    AttributeInfo(
        name="Description",
        description="Description of the product", type="string"
    ),
    AttributeInfo(
        name="Features",
        description="Features of the product",
        type="string"
    ),
]
document_content_description = "Product listing"

In [19]:
# !pip install lark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.6/111.6 kB 1.7 MB/s eta 0:00:00


In [ ]:
# Configure retriver
llm = OpenAI(temperature=0)
retriever = SelfQueryRetriever.from_llm(llm, db, document_content_description, metadata_field_info, verbose=True)

# Retrieve values
retriever.get_relevant_documents("good heart monitor")

# The result is:
# Description: Achieve your fitness goals with our Fitness Tracker Smartwatch. Monitor your activity, heart rate, and receive notifications on your wrist.\n
# Features: - Heart rate monitor and activity tracking.<br> - Built-in GPS for accurate workout tracking.<br> - Sleep analysis and guided breathing exercises.<br> - Receive notifications from your smartphone.<br> - Water-resistant for workouts and everyday use.<br> - Long battery life.',
# metadata={
#    'Product Name': 'Fitness Tracker Smartwatch',
#    'Price': '149.99',
#    'Rating': '4.5',
#    'Description': 'Achieve your fitness goals with our Fitness Tracker Smartwatch. Monitor your activity, heart rate, and receive notifications on your wrist.',
#    'Features': '- Heart rate monitor and activity tracking.<br> - Built-in GPS for accurate workout tracking.<br> - Sleep analysis and guided breathing exercises.<br> - Receive notifications from your smartphone.<br> - Water-resistant for workouts and everyday use.<br> - Long battery life.'
# }